# Cell Segmentation
This notebook demonstrates a workflow for segmenting cells in a microscopy image using background subtraction, marker-based watershed algorithm, and measurement of segmented areas.

## Step 1: Load the Image
We begin by loading the fluorescence microscopy image of cells which we want to segment.

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

image = io.imread('../data/membranes.png')
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.show()

## Step 2: Convert to Grayscale
Depending on the image format, it might be necessary to convert it to grayscale.

In [ ]:
from skimage.color import rgb2gray

if len(image.shape) == 3:
    image_gray = rgb2gray(image)
else:
    image_gray = image

## Step 3: Background Subtraction using White Top-Hat
Apply white top-hat transformation to mitigate uneven illumination in the image.

In [ ]:
from skimage.morphology import disk, white_tophat

selem = disk(10)
image_tophat = white_tophat(image_gray, selem)

plt.imshow(image_tophat, cmap='gray')
plt.title('Background Subtracted')
plt.show()

## Step 4: Compute Gradient
Utilize the Sobel filter to calculate image gradients, highlighting edges.

In [ ]:
from skimage.filters import sobel

gradient = sobel(image_tophat)
plt.imshow(gradient, cmap='gray')
plt.title('Gradient Image')
plt.show()

## Step 5: Enhance Marker Generation
Create seed markers for the watershed algorithm based on local maxima in the distance-transformed image.

In [ ]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max

distance = ndi.distance_transform_edt(image_tophat)
local_maxi = peak_local_max(distance, indices=False, min_distance=5)
markers = ndi.label(local_maxi)[0]

plt.imshow(markers, cmap='nipy_spectral')
plt.title('Markers')
plt.show()

## Step 6: Apply Watershed Algorithm
Segment the image using the watershed algorithm with the generated markers.

In [ ]:
from skimage.segmentation import watershed

labeled_image = watershed(gradient, markers)

plt.imshow(labeled_image, cmap='nipy_spectral')
plt.title('Watershed Segmentation')
plt.show()

## Step 7: Remove Small Objects
Clean up the segmentation by removing regions smaller than a specified size.

In [ ]:
from skimage.morphology import remove_small_objects

cleaned_image = remove_small_objects(labeled_image, min_size=150)

plt.imshow(cleaned_image, cmap='nipy_spectral')
plt.title('Cleaned Segmentation')
plt.show()

## Step 8: Measure Segmented Cell Areas
Measure the area of each segmented cell and display the results.

In [ ]:
from skimage.measure import regionprops_table

properties = regionprops_table(cleaned_image, properties=['label', 'area'])
import pandas as pd
df = pd.DataFrame(properties)
print(df)